In [ ]:
import torch
import numpy as np
import torch.nn as nn
import joblib
import pandas as pd
import random
import os


# Create neural network layers

In [ ]:
class Regressor(nn.Module):
    def __init__(self, input_dim, units,num_layers):
        super().__init__()
        layers=[]
        in_dim=input_dim


        for i in range(num_layers):
            out_dim=units*2**(num_layers-(i+1))
            layers.append(nn.Linear(in_dim,out_dim))
            layers.append(nn.BatchNorm1d(out_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))
            in_dim=out_dim
        layers.append(nn.Linear(in_dim,1))
        self.net = nn.Sequential(*layers)

    def forward(self,x):
        return self.net(x)

# Load saved pytorch model and saved test data files

In [ ]:
checkpoint = torch.load('best_model_complete_pt.pth')
preprocessor = joblib.load("preprocessor.pkl")
Test_data_enc = pd.read_csv("Test_data_enc.csv")
X_test_pre = preprocessor.transform(Test_data_enc)

# Extract neural network weights, neural network  layers parameters and input tensor data

In [ ]:
X_test_tensor=torch.tensor(X_test_pre.todense(), dtype=torch.float32)
input_dim = checkpoint['model_params']['input_dim']
units = checkpoint['model_params']['units']
num_layers = checkpoint['model_params']['num_layers']

# Make predictions

In [ ]:
model = Regressor(input_dim, units, num_layers)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor)

y_pred_test = y_pred_test.flatten()
Prediction=pd.Series(y_pred_test.tolist())
Prediction.name = 'Predicted Price'
Prediction=pd.DataFrame(Prediction)
Prediction['Predicted Price'] = Prediction['Predicted Price'].astype(int)
print(Prediction)


     Predicted Price
0              15664
1              28875
2              10522
3              18817
4              30857
..               ...
995            34434
996            48921
997            33018
998            36891
999            57759

[1000 rows x 1 columns]
